In [4]:
#Dependencies
import matplotlib.pyplot as plt
import requests
from pprint import pprint
from scipy import stats
import pandas as pd
import numpy as np
from api_keys import api_key
import json

In [5]:
# !pip install sodapy

In [6]:
from sodapy import Socrata

client = Socrata("datacatalog.cookcountyil.gov", "a5adtglEZdwTXbUuRvAvPXv59")

ModuleNotFoundError: No module named 'sodapy'

In [ ]:
# zip_df = clean_zipcode_response(results)
# zip_df.head(10)

In [ ]:
#List of zipcodes we want to look at

zipcodes = [60618, 60647, 60625, 60608, 60624, 60614]


In [ ]:
#Declare function

def clean_zipcode_response(api_response):
    df = pd.DataFrame(api_response)
    df["year"] = df["year"].astype('int')
    df_limited_cols = df[["prop_address_zipcode_1", "pin", "year"]]
    return df_limited_cols

In [ ]:
final_zip = pd.DataFrame(columns = ["prop_address_zipcode_1", "pin", "year"])

for zipcode in zipcodes:
    universe_response = .geclientt("tx2p-k2g9",prop_address_zipcode_1={zipcode}, where="year > 2015", limit=20000) #50,000
    df_zip = clean_zipcode_response(universe_response)
    final_zip = final_zip.append(df_zip)
final_zip.head(10)
# final_zip.to_csv("final_zip.csv", encoding='utf-8', index=False)

In [ ]:
def clean_pin_response(api_response):
    df = pd.DataFrame(api_response)
    if "year" not in df:
        return pd.DataFrame(columns = ["pin", "year", "board_tot"])
    df["year"] = df["year"].astype('int')
    if "board_tot" not in df:
        return pd.DataFrame(columns = ["pin", "year", "board_tot"])
    df_limited_cols = df[["pin", "year", "board_tot"]]
    df_limited_cols = df_limited_cols.dropna()
    df_final = df_limited_cols[df_limited_cols["year"] > 2015]
    return df_final

In [ ]:
pins = final_zip.pin.unique()

#board_tot is total assessed value 

# final_pin = pd.DataFrame(columns = ["pin", "year", "board_tot"])

for pin in remaining_pins:
    historic_url = f"https://datacatalog.cookcountyil.gov/resource/uzyt-m557.json?pin={pin}"
    historic_response = requests.get(historic_url).json()
    columns = ["pin", "year", "board_tot"]
    final_pin = final_pin.reindex(columns=columns)
    df_pin = clean_pin_response(historic_response)
    final_pin = final_pin.append(df_pin)
final_pin.head(10)

In [ ]:
merged_df = pd.merge(final_zip, final_pin, on=["pin", "year"])
merged_df.head(10)

In [ ]:
modified_df = merged_df
modified_df['board_tot_num'] = modified_df['board_tot'].astype('int')
mean_zip = modified_df.groupby(['prop_address_zipcode_1', 'year'])['board_tot_num'].mean()
mean_zip.head(30)

In [ ]:
print(final_zip['pin'].nunique())
print(final_pin['pin'].nunique())
remaining_pins = list(set(final_zip.pin) - set(final_pin.pin))
print(len(remaining_pins))

In [ ]:
final_zip.to_csv("final_zip.csv", encoding='utf-8', index=False)
final_pin.to_csv("final_pin.csv", encoding='utf-8', index=False)
merged_df.to_csv("merged_df.csv", encoding='utf-8', index=False)

In [ ]:
df_60618 = merged_df.loc[merged_df["prop_address_zipcode_1"] == "60618"]
df_60647 = merged_df.loc[merged_df["prop_address_zipcode_1"] == "60647"]
df_60625 = merged_df.loc[merged_df["prop_address_zipcode_1"] == "60625"]
df_60608 = merged_df.loc[merged_df["prop_address_zipcode_1"] == "60608"]
df_60624 = merged_df.loc[merged_df["prop_address_zipcode_1"] == "60624"]
df_60614 = merged_df.loc[merged_df["prop_address_zipcode_1"] == "60614"]

In [ ]:
#60618

df_60618_final = df_60618["board_tot_num"]

df_60618_quartiles = df_60618_final.quantile([.25,.5,.75])
df_60618_low_quartile = df_60618_quartiles[.25]
df_60618_high_quartile = df_60618_quartiles[.75]
df_60618_mean = df_60618_quartiles.mean()
df_60618_iqr = df_60618_high_quartile-df_60618_low_quartile
df_60618_lower_bound = df_60618_low_quartile - (1.5*df_60618_iqr)
df_60618_upper_bound = df_60618_high_quartile + (1.5*df_60618_iqr)
df_60618_outliers = df_60618_quartiles.loc[(df_60618_quartiles < df_60618_lower_bound) & (df_60618_quartiles > df_60618_upper_bound)]

print(f"60618 Low Quartile: {df_60618_low_quartile}")
print(f"60618 Mean: {df_60618_mean}")
print(f"60618 High Quartile: {df_60618_high_quartile}")
print(f"60618 Outliers: Below {df_60618_lower_bound} And Above {df_60618_upper_bound}")



In [ ]:
#60647

df_60647_final = df_60647["board_tot_num"]

df_60647_quartiles = df_60647_final.quantile([.25,.5,.75])
df_60647_low_quartile = df_60647_quartiles[.25]
df_60647_high_quartile = df_60647_quartiles[.75]
df_60647_mean = df_60647_quartiles.mean()
df_60647_iqr = df_60618_high_quartile-df_60647_low_quartile
df_60647_lower_bound = df_60647_low_quartile - (1.5*df_60647_iqr)
df_60647_upper_bound = df_60647_high_quartile + (1.5*df_60647_iqr)
df_60647_outliers = df_60647_quartiles.loc[(df_60618_quartiles < df_60647_lower_bound) & (df_60647_quartiles > df_60647_upper_bound)]

print(f"60647 Low Quartile: {df_60647_low_quartile}")
print(f"60647 Mean: {df_60647_mean}")
print(f"60647 High Quartile: {df_60647_high_quartile}")
print(f"60647 Outliers: Below {df_60647_lower_bound} And Above {df_60647_upper_bound}")


In [ ]:
#60625

df_60625_final = df_60625["board_tot_num"]

df_60625_quartiles = df_60625_final.quantile([.25,.5,.75])
df_60625_low_quartile = df_60625_quartiles[.25]
df_60625_high_quartile = df_60625_quartiles[.75]
df_60625_mean = df_60625_quartiles.mean()
df_60625_iqr = df_60625_high_quartile-df_60625_low_quartile
df_60625_lower_bound = df_60625_low_quartile - (1.5*df_60625_iqr)
df_60625_upper_bound = df_60625_high_quartile + (1.5*df_60625_iqr)
df_60625_outliers = df_60625_quartiles.loc[(df_60625_quartiles < df_60625_lower_bound) & (df_60625_quartiles > df_60625_upper_bound)]

print(f"60625 Low Quartile: {df_60625_low_quartile}")
print(f"60625 Mean: {df_60625_mean}")
print(f"60625 High Quartile: {df_60625_high_quartile}")
print(f"60625 Outliers: Below {df_60625_lower_bound} And Above {df_60625_upper_bound}")


In [ ]:
#60608

df_60608_final = df_60608["board_tot_num"]

df_60608_quartiles = df_60608_final.quantile([.25,.5,.75])
df_60608_low_quartile = df_60608_quartiles[.25]
df_60608_high_quartile = df_60608_quartiles[.75]
df_60608_mean = df_60608_quartiles.mean()
df_60608_iqr = df_60608_high_quartile-df_60608_low_quartile
df_60608_lower_bound = df_60608_low_quartile - (1.5*df_60608_iqr)
df_60608_upper_bound = df_60608_high_quartile + (1.5*df_60608_iqr)
df_60608_outliers = df_60608_quartiles.loc[(df_60608_quartiles < df_60608_lower_bound) & (df_60608_quartiles > df_60608_upper_bound)]

print(f"60608 Low Quartile: {df_60608_low_quartile}")
print(f"60608 Mean: {df_60608_mean}")
print(f"60608 High Quartile: {df_60608_high_quartile}")
print(f"60608 Outliers: Below {df_60608_lower_bound} And Above {df_60608_upper_bound}")


In [ ]:
#60624

df_60624_final = df_60624["board_tot_num"]

df_60624_quartiles = df_60624_final.quantile([.25,.5,.75])
df_60624_low_quartile = df_60624_quartiles[.25]
df_60624_high_quartile = df_60624_quartiles[.75]
df_60624_mean = df_60624_quartiles.mean()
df_60624_iqr = df_60624_high_quartile-df_60624_low_quartile
df_60624_lower_bound = df_60624_low_quartile - (1.5*df_60624_iqr)
df_60624_upper_bound = df_60624_high_quartile + (1.5*df_60624_iqr)
df_60624_outliers = df_60624_quartiles.loc[(df_60624_quartiles < df_60624_lower_bound) & (df_60624_quartiles > df_60624_upper_bound)]

print(f"60624 Low Quartile: {df_60624_low_quartile}")
print(f"60624 Mean: {df_60624_mean}")
print(f"60624 High Quartile: {df_60624_high_quartile}")
print(f"60624 Outliers: Below {df_60624_lower_bound} And Above {df_60624_upper_bound}")



In [ ]:
#60614

df_60614_final = df_60614["board_tot_num"]

df_60614_quartiles = df_60614_final.quantile([.25,.5,.75])
df_60614_low_quartile = df_60614_quartiles[.25]
df_60614_high_quartile = df_60614_quartiles[.75]
df_60614_mean = df_60614_quartiles.mean()
df_60614_iqr = df_60614_high_quartile-df_60614_low_quartile
df_60614_lower_bound = df_60614_low_quartile - (1.5*df_60614_iqr)
df_60614_upper_bound = df_60614_high_quartile + (1.5*df_60614_iqr)
df_60614_outliers = df_60614_quartiles.loc[(df_60614_quartiles < df_60614_lower_bound) & (df_60614_quartiles > df_60614_upper_bound)]

print(f"60614 Low Quartile: {df_60614_low_quartile}")
print(f"60614 Mean: {df_60614_mean}")
print(f"60614 High Quartile: {df_60614_high_quartile}")
print(f"60614 Outliers: Below {df_60614_lower_bound} And Above {df_60614_upper_bound}")




In [ ]:
data = [df_60608_final, df_60614_final, df_60618_final, df_60624_final, df_60625_final, df_60647_final]
zipcodes = ["60608", "60614", "60618", "60624", "60625", "60647"]
figl, axl = plt.subplots()
axl.set_title("Assessed Values Prices by Zipcode")
axl.set_ylabel("Assessed Values")
axl.set_xlabel("Zipcodes")
ymin = 0
ymax = 90000
plt.ylim(ymin, ymax)
axl.boxplot(data, labels=zipcodes)
plt.show()

In [ ]:
#Declare function

def clean_zipcode_response(api_response):
    df = pd.DataFrame(api_response)
    df["year"] = df["year"].astype('int')
    df_limited_cols = df[["prop_address_zipcode_1", "pin", "year"]]
    return df_limited_cols

In [ ]:
final_zip = pd.DataFrame(columns = ["prop_address_zipcode_1", "pin", "year"])

for zipcode in zipcodes:
    universe_response = .geclientt("tx2p-k2g9",prop_address_zipcode_1={zipcode}, where="year > 2015", limit=20000) #50,000
    df_zip = clean_zipcode_response(universe_response)
    final_zip = final_zip.append(df_zip)
final_zip.head(10)
# final_zip.to_csv("final_zip.csv", encoding='utf-8', index=False)

In [ ]:
def clean_pin_response(api_response):
    df = pd.DataFrame(api_response)
    if "year" not in df:
        return pd.DataFrame(columns = ["pin", "year", "board_tot"])
    df["year"] = df["year"].astype('int')
    if "board_tot" not in df:
        return pd.DataFrame(columns = ["pin", "year", "board_tot"])
    df_limited_cols = df[["pin", "year", "board_tot"]]
    df_limited_cols = df_limited_cols.dropna()
    df_final = df_limited_cols[df_limited_cols["year"] > 2015]
    return df_final

In [ ]:
pins = final_zip.pin.unique()

#board_tot is total assessed value 

# final_pin = pd.DataFrame(columns = ["pin", "year", "board_tot"])

for pin in remaining_pins:
    historic_url = f"https://datacatalog.cookcountyil.gov/resource/uzyt-m557.json?pin={pin}"
    historic_response = requests.get(historic_url).json()
    columns = ["pin", "year", "board_tot"]
    final_pin = final_pin.reindex(columns=columns)
    df_pin = clean_pin_response(historic_response)
    final_pin = final_pin.append(df_pin)
final_pin.head(10)

In [ ]:
merged_df = pd.merge(final_zip, final_pin, on=["pin", "year"])
merged_df.head(10)

In [ ]:
modified_df = merged_df
modified_df['board_tot_num'] = modified_df['board_tot'].astype('int')
mean_zip = modified_df.groupby(['prop_address_zipcode_1', 'year'])['board_tot_num'].mean()
mean_zip.head(30)

In [ ]:
print(final_zip['pin'].nunique())
print(final_pin['pin'].nunique())
remaining_pins = list(set(final_zip.pin) - set(final_pin.pin))
print(len(remaining_pins))

In [ ]:
final_zip.to_csv("final_zip.csv", encoding='utf-8', index=False)
final_pin.to_csv("final_pin.csv", encoding='utf-8', index=False)
merged_df.to_csv("merged_df.csv", encoding='utf-8', index=False)

In [ ]:
df_60618 = merged_df.loc[merged_df["prop_address_zipcode_1"] == "60618"]
df_60647 = merged_df.loc[merged_df["prop_address_zipcode_1"] == "60647"]
df_60625 = merged_df.loc[merged_df["prop_address_zipcode_1"] == "60625"]
df_60608 = merged_df.loc[merged_df["prop_address_zipcode_1"] == "60608"]
df_60624 = merged_df.loc[merged_df["prop_address_zipcode_1"] == "60624"]
df_60614 = merged_df.loc[merged_df["prop_address_zipcode_1"] == "60614"]

In [ ]:
#60618

df_60618_final = df_60618["board_tot_num"]

df_60618_quartiles = df_60618_final.quantile([.25,.5,.75])
df_60618_low_quartile = df_60618_quartiles[.25]
df_60618_high_quartile = df_60618_quartiles[.75]
df_60618_mean = df_60618_quartiles.mean()
df_60618_iqr = df_60618_high_quartile-df_60618_low_quartile
df_60618_lower_bound = df_60618_low_quartile - (1.5*df_60618_iqr)
df_60618_upper_bound = df_60618_high_quartile + (1.5*df_60618_iqr)
df_60618_outliers = df_60618_quartiles.loc[(df_60618_quartiles < df_60618_lower_bound) & (df_60618_quartiles > df_60618_upper_bound)]

print(f"60618 Low Quartile: {df_60618_low_quartile}")
print(f"60618 Mean: {df_60618_mean}")
print(f"60618 High Quartile: {df_60618_high_quartile}")
print(f"60618 Outliers: Below {df_60618_lower_bound} And Above {df_60618_upper_bound}")



In [ ]:
#60647

df_60647_final = df_60647["board_tot_num"]

df_60647_quartiles = df_60647_final.quantile([.25,.5,.75])
df_60647_low_quartile = df_60647_quartiles[.25]
df_60647_high_quartile = df_60647_quartiles[.75]
df_60647_mean = df_60647_quartiles.mean()
df_60647_iqr = df_60618_high_quartile-df_60647_low_quartile
df_60647_lower_bound = df_60647_low_quartile - (1.5*df_60647_iqr)
df_60647_upper_bound = df_60647_high_quartile + (1.5*df_60647_iqr)
df_60647_outliers = df_60647_quartiles.loc[(df_60618_quartiles < df_60647_lower_bound) & (df_60647_quartiles > df_60647_upper_bound)]

print(f"60647 Low Quartile: {df_60647_low_quartile}")
print(f"60647 Mean: {df_60647_mean}")
print(f"60647 High Quartile: {df_60647_high_quartile}")
print(f"60647 Outliers: Below {df_60647_lower_bound} And Above {df_60647_upper_bound}")


In [ ]:
#60625

df_60625_final = df_60625["board_tot_num"]

df_60625_quartiles = df_60625_final.quantile([.25,.5,.75])
df_60625_low_quartile = df_60625_quartiles[.25]
df_60625_high_quartile = df_60625_quartiles[.75]
df_60625_mean = df_60625_quartiles.mean()
df_60625_iqr = df_60625_high_quartile-df_60625_low_quartile
df_60625_lower_bound = df_60625_low_quartile - (1.5*df_60625_iqr)
df_60625_upper_bound = df_60625_high_quartile + (1.5*df_60625_iqr)
df_60625_outliers = df_60625_quartiles.loc[(df_60625_quartiles < df_60625_lower_bound) & (df_60625_quartiles > df_60625_upper_bound)]

print(f"60625 Low Quartile: {df_60625_low_quartile}")
print(f"60625 Mean: {df_60625_mean}")
print(f"60625 High Quartile: {df_60625_high_quartile}")
print(f"60625 Outliers: Below {df_60625_lower_bound} And Above {df_60625_upper_bound}")


In [ ]:
#60608

df_60608_final = df_60608["board_tot_num"]

df_60608_quartiles = df_60608_final.quantile([.25,.5,.75])
df_60608_low_quartile = df_60608_quartiles[.25]
df_60608_high_quartile = df_60608_quartiles[.75]
df_60608_mean = df_60608_quartiles.mean()
df_60608_iqr = df_60608_high_quartile-df_60608_low_quartile
df_60608_lower_bound = df_60608_low_quartile - (1.5*df_60608_iqr)
df_60608_upper_bound = df_60608_high_quartile + (1.5*df_60608_iqr)
df_60608_outliers = df_60608_quartiles.loc[(df_60608_quartiles < df_60608_lower_bound) & (df_60608_quartiles > df_60608_upper_bound)]

print(f"60608 Low Quartile: {df_60608_low_quartile}")
print(f"60608 Mean: {df_60608_mean}")
print(f"60608 High Quartile: {df_60608_high_quartile}")
print(f"60608 Outliers: Below {df_60608_lower_bound} And Above {df_60608_upper_bound}")


In [ ]:
#60624

df_60624_final = df_60624["board_tot_num"]

df_60624_quartiles = df_60624_final.quantile([.25,.5,.75])
df_60624_low_quartile = df_60624_quartiles[.25]
df_60624_high_quartile = df_60624_quartiles[.75]
df_60624_mean = df_60624_quartiles.mean()
df_60624_iqr = df_60624_high_quartile-df_60624_low_quartile
df_60624_lower_bound = df_60624_low_quartile - (1.5*df_60624_iqr)
df_60624_upper_bound = df_60624_high_quartile + (1.5*df_60624_iqr)
df_60624_outliers = df_60624_quartiles.loc[(df_60624_quartiles < df_60624_lower_bound) & (df_60624_quartiles > df_60624_upper_bound)]

print(f"60624 Low Quartile: {df_60624_low_quartile}")
print(f"60624 Mean: {df_60624_mean}")
print(f"60624 High Quartile: {df_60624_high_quartile}")
print(f"60624 Outliers: Below {df_60624_lower_bound} And Above {df_60624_upper_bound}")



In [ ]:
#60614

df_60614_final = df_60614["board_tot_num"]

df_60614_quartiles = df_60614_final.quantile([.25,.5,.75])
df_60614_low_quartile = df_60614_quartiles[.25]
df_60614_high_quartile = df_60614_quartiles[.75]
df_60614_mean = df_60614_quartiles.mean()
df_60614_iqr = df_60614_high_quartile-df_60614_low_quartile
df_60614_lower_bound = df_60614_low_quartile - (1.5*df_60614_iqr)
df_60614_upper_bound = df_60614_high_quartile + (1.5*df_60614_iqr)
df_60614_outliers = df_60614_quartiles.loc[(df_60614_quartiles < df_60614_lower_bound) & (df_60614_quartiles > df_60614_upper_bound)]

print(f"60614 Low Quartile: {df_60614_low_quartile}")
print(f"60614 Mean: {df_60614_mean}")
print(f"60614 High Quartile: {df_60614_high_quartile}")
print(f"60614 Outliers: Below {df_60614_lower_bound} And Above {df_60614_upper_bound}")




In [ ]:
data = [df_60608_final, df_60614_final, df_60618_final, df_60624_final, df_60625_final, df_60647_final]
zipcodes = ["60608", "60614", "60618", "60624", "60625", "60647"]
figl, axl = plt.subplots()
axl.set_title("Assessed Values Prices by Zipcode")
axl.set_ylabel("Assessed Values")
axl.set_xlabel("Zipcodes")
ymin = 0
ymax = 90000
plt.ylim(ymin, ymax)
axl.boxplot(data, labels=zipcodes)
plt.show()

In [ ]:
data = [df_60625_final, df_60608_final, df_60618_final, df_60647_final, df_60624_final, df_60614_final]
zipcodes = ["60625", "60608", "60618", "60647", "60624", "60614"]
figl, axl = plt.subplots()
axl.set_title("Assessed Values Prices by Zipcode")
axl.set_ylabel("Assessed Values")
axl.set_xlabel("Zipcodes")
ymin = 0
ymax = 90000
plt.ylim(ymin, ymax)
axl.boxplot(data, labels=zipcodes)
plt.show()

In [ ]:
# #ANOVA with the skew to mimic 
# data = [df_60625_final, df_60608_final, df_60618_final, df_60647_final, df_60624_final, df_60614_final]
# zipcodes = ["60625", "60608", "60618", "60647", "60624", "60614"]
# figl, axl = plt.subplots()
# axl.set_title("Assessed Values Prices by Zipcode")
# axl.set_ylabel("Assessed Values")
# axl.set_xlabel("Zipcodes")
# ymin = 0
# ymax = 90000
# plt.ylim(ymin, ymax)
# axl.boxplot(data, labels=zipcodes)
# plt.show()
